# Homework 3: Classification, Regression, and Other Prediction Models

## Task 1

Round DATETIME towards its nearest hour. (e.g. 2017/10/16 14:37:52 -> 2017/10/16 15:00:00)

```sql
UPDATE 逐時觀測
SET 逐時觀測.時間 = DATE_FORMAT(
  DATE_ADD(逐時觀測.時間, INTERVAL 30 MINUTE),
  '%Y-%m-%d %H:00:00'
)
```

```sql
UPDATE Power
SET Power.updateTime = DATE_FORMAT(
  DATE_ADD(Power.updateTime, INTERVAL 30 MINUTE),
  '%Y-%m-%d %H:00:00'
)
```

A naive way to generate dataset.

```sql
DROP PROCEDURE IF EXISTS dm.generate_dataset;
DELIMITER //
CREATE PROCEDURE dm.generate_dataset(from_time DATETIME, to_time DATETIME)
BEGIN
  DECLARE it INT;
  DECLARE temp DOUBLE;
  DECLARE output TEXT;
  DECLARE curr_time DATETIME;
  WHILE from_time <= to_time DO
    SET it = 0;
    SET output = '';
    SET curr_time = from_time;
    generate_record: WHILE it < 102 DO
      SET temp = NULL;
      IF it < 96 THEN
        SELECT 逐時觀測.溫度 INTO temp FROM 逐時觀測
          WHERE (逐時觀測.時間 = curr_time) AND (逐時觀測.測站 = 'BANQIAO,板橋')
          LIMIT 1;
      ELSE
        SELECT Power.northUsage INTO temp FROM Power
          WHERE (Power.updateTime = curr_time)
          LIMIT 1;
      END IF;
      IF temp IS NULL THEN
        SET output = '';
        LEAVE generate_record;
      END IF;
      SET it = it + 1;
      SET output = CONCAT(output, ',', CAST(temp AS CHAR));
      SET curr_time = DATE_ADD(curr_time, INTERVAL 1 HOUR);
    END WHILE;
    IF output <> '' THEN
      INSERT INTO Records VALUES (output);
    END IF;
    SET from_time = DATE_ADD(from_time, INTERVAL 1 HOUR);
  END WHILE;
END//
DELIMITER ;
SET @from_time = CAST('2016-07-03 01:00:00' AS DATETIME);
SET @to_time = CAST('2017-07-04 00:00:00' AS DATETIME);
CALL generate_dataset(@from_time, @to_time);
```

This is very inefficient because there are too many queries. Try a better approach.

First, find duplicate record times.

```sql
SELECT 逐時觀測.時間
FROM 逐時觀測
WHERE 逐時觀測.測站 = 'BANQIAO,板橋'
GROUP BY 逐時觀測.時間
HAVING COUNT(*) > 1
```

0 rows retrieved.

```sql
SELECT Power.updateTime, COUNT(*) AS cnt
FROM Power
GROUP BY Power.updateTime
HAVING cnt > 1
```

updateTime | cnt
--- | ---
2016-10-17 00:00:00 | 2
2017-05-20 08:00:00 | 3
2017-05-20 13:00:00 | 2
2017-05-30 22:00:00 | 2
2017-07-19 09:00:00 | 2
2017-07-24 01:00:00 | 2
2017-07-24 13:00:00 | 2
2017-08-21 20:00:00 | 2
2017-08-21 21:00:00 | 2

Remove them using the following query.

```sql
-- run 10 times
DELETE FROM Power
WHERE Power.updateTime IN (
  SELECT A.updateTime
  FROM (SELECT * FROM Power) AS A
  GROUP BY A.updateTime
  HAVING COUNT(*) > 1
) LIMIT 1
```

Store each record (96 history data and 6 prediction data) in one CSV file.  
Ignore the whole record if it contains missing data.

```sql
DROP PROCEDURE IF EXISTS dm.generate_dataset;
DELIMITER //
CREATE PROCEDURE dm.generate_dataset(from_time DATETIME, to_time DATETIME)
BEGIN
  DECLARE curr_time DATETIME DEFAULT from_time;
  SET @path = '/Users/ywpu/Repositories/data-mining/hw3/';
  generate_record: WHILE curr_time <= to_time DO
    SET @cnt1 = (
      SELECT COUNT(逐時觀測.溫度) FROM 逐時觀測
        WHERE (逐時觀測.時間 >= curr_time)
        AND (逐時觀測.時間 < DATE_ADD(curr_time, INTERVAL 4 DAY))
        AND (逐時觀測.測站 = 'BANQIAO,板橋')
    );
    SET @cnt2 = (
      SELECT COUNT(Power.northUsage) FROM Power
        WHERE (Power.updateTime >= DATE_ADD(curr_time, INTERVAL 4 DAY))
        AND (Power.updateTime < DATE_ADD(curr_time, INTERVAL '4 6' DAY_HOUR))
    );
    IF @cnt1 <> 96 OR @cnt2 <> 6 THEN
      SET curr_time = DATE_ADD(curr_time, INTERVAL 1 HOUR);
      ITERATE generate_record;
    END IF;
    SET @filename = CONCAT(@path, CAST(curr_time AS CHAR), '.csv');
    SET @query_string = CONCAT(
      "SELECT A.Y INTO OUTFILE '", @filename, "'
        FIELDS TERMINATED BY ',' LINES TERMINATED BY '\n'
      FROM (
        (
          SELECT 逐時觀測.時間 AS X, 逐時觀測.溫度 AS Y FROM 逐時觀測
          WHERE (逐時觀測.時間 >= ?)
          AND (逐時觀測.時間 < DATE_ADD(?, INTERVAL 4 DAY))
          AND (逐時觀測.測站 = 'BANQIAO,板橋')
        ) UNION ALL (
          SELECT Power.updateTime AS X, Power.northUsage AS Y FROM Power
          WHERE (Power.updateTime >= DATE_ADD(?, INTERVAL 4 DAY))
          AND (Power.updateTime < DATE_ADD(?, INTERVAL '4 6' DAY_HOUR))
        ) ORDER BY X ASC
      ) A"
    );
    PREPARE stmt FROM @query_string;
    EXECUTE stmt USING curr_time, curr_time, curr_time, curr_time;
    DEALLOCATE PREPARE stmt;
    SET curr_time = DATE_ADD(curr_time, INTERVAL 1 HOUR);
  END WHILE;
END//
DELIMITER ;
SET @from_time = CAST('2016-07-03 01:00:00' AS DATETIME);
SET @to_time = CAST('2017-07-04 00:00:00' AS DATETIME);
CALL generate_dataset(@from_time, @to_time);
```

Combine all records into `dataset.csv`.

In [3]:
import os
with open('dataset.csv', 'w') as target:
    entries = sorted(os.listdir())
    for entry in entries:
        if entry.endswith('.csv') and entry.startswith('201'):
            with open(entry) as file:
                cols = file.readlines()
            cols = [col.strip() for col in cols]
            row = ','.join(cols) + '\n'
            target.write(row)

Load `dataset.csv` into DataFrame.

In [4]:
import numpy as np
import pandas as pd
dataset = pd.read_csv('dataset.csv', names=list(range(0, 102)), dtype=np.float64)

In [6]:
dataset.shape

(4568, 102)

In [7]:
dataset.head()

,0,1,2,3,4,5,6,7,8,9,...,92,93,94,95,96,97,98,99,100,101
0,27.6,28.4,29.4,29.7,28.3,27.1,26.3,25.9,25.6,25.6,...,27.3,27.4,27.4,27.7,841.3,826.4,789.5,773.6,778.0,783.1
1,28.4,29.4,29.7,28.3,27.1,26.3,25.9,25.6,25.6,25.4,...,27.4,27.4,27.7,26.6,826.4,789.5,773.6,778.0,783.1,814.3
2,29.4,29.7,28.3,27.1,26.3,25.9,25.6,25.6,25.4,25.3,...,27.4,27.7,26.6,26.0,789.5,773.6,778.0,783.1,814.3,812.4
3,29.7,28.3,27.1,26.3,25.9,25.6,25.6,25.4,25.3,25.3,...,27.7,26.6,26.0,25.9,773.6,778.0,783.1,814.3,812.4,809.0
4,28.3,27.1,26.3,25.9,25.6,25.6,25.4,25.3,25.3,25.3,...,26.6,26.0,25.9,25.8,778.0,783.1,814.3,812.4,809.0,796.6


## Task 2

Split dataset randomly using scikit-learn.

In [8]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(dataset, test_size=0.3)

In [9]:
train.shape

(3197, 102)

In [10]:
train.head()

,0,1,2,3,4,5,6,7,8,9,...,92,93,94,95,96,97,98,99,100,101
4228,22.6,22.5,22.6,22.6,22.5,22.6,22.7,22.8,22.8,23.2,...,27.5,26.9,26.6,26.4,985.5,956.5,863.9,815.0,793.3,754.1
275,31.9,32.0,32.4,32.4,31.3,30.1,29.1,28.7,28.6,28.5,...,22.5,23.0,23.2,23.1,860.9,869.8,864.0,871.3,874.2,878.7
1696,21.3,21.0,20.7,20.6,20.5,20.3,20.4,20.0,20.1,19.9,...,21.1,21.3,21.1,21.2,780.8,718.8,694.2,665.2,659.2,670.3
2621,13.8,13.7,13.2,12.9,12.9,12.8,12.8,12.8,13.1,13.7,...,19.7,18.3,18.2,18.0,709.1,684.2,679.7,699.5,722.2,785.7
944,21.2,21.2,20.9,20.9,21.1,21.2,21.1,21.2,21.1,21.4,...,25.6,25.3,24.0,23.4,863.9,805.2,757.8,720.4,711.3,689.3


In [11]:
test.shape

(1371, 102)

In [12]:
test.head()

,0,1,2,3,4,5,6,7,8,9,...,92,93,94,95,96,97,98,99,100,101
2757,18.8,18.3,17.7,17.7,17.7,17.9,17.0,16.5,16.2,15.3,...,15.1,14.8,14.8,14.5,990.8,982.2,960.6,939.8,919.2,876.8
2887,23.9,23.6,23.1,23.2,23.0,23.7,25.6,26.4,26.6,26.1,...,20.1,20.1,20.1,20.2,624.3,619.4,609.2,631.6,648.5,705.1
3234,30.9,29.8,29.5,28.7,28.7,28.0,27.8,27.6,27.2,26.2,...,27.4,29.0,30.5,30.2,1157.5,1169.0,1167.8,1156.8,1122.0,1119.9
50,26.3,26.2,26.1,25.9,25.6,26.4,27.2,28.0,29.3,29.9,...,27.1,26.5,26.2,25.3,817.8,797.7,782.7,782.2,792.0,839.4
3854,26.6,27.2,27.6,30.2,31.5,33.0,33.1,33.6,33.1,31.0,...,24.0,23.9,24.1,24.3,762.3,815.2,877.3,907.0,914.6,913.2
